In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from peft import AutoPeftModelForTokenClassification
from datasets import load_dataset
from glob import glob

In [2]:
dataset = load_dataset("clarin-knext/wsd_polish_datasets", trust_remote_code=True)

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'tokens', 'phrases', 'wsd'],
        num_rows: 7848
    })
})

In [4]:
dataset['train'].features['tokens'].feature['pos']

Value(dtype='string', id=None)

In [5]:
tokenizer_bert = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model_bert = AutoModelForTokenClassification.from_pretrained("allegro/herbert-base-cased")

tokenizer_gpt = AutoTokenizer.from_pretrained("sdadas/polish-gpt2-medium", add_prefix_space=True)
tokenizer_gpt.pad_token = tokenizer_gpt.eos_token
model_gpt = AutoModelForTokenClassification.from_pretrained("sdadas/polish-gpt2-medium", pad_token_id=tokenizer_gpt.pad_token_id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at sdadas/polish-gpt2-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
sentence = dataset['train'][0]['text']

In [7]:
def get_embeddings(text, tokenizer, model, layer=-1):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs,output_hidden_states=True)
    # print(outputs)
    # x = outputs.hidden_states[-1][:, 0, :].cpu().detach().numpy().squeeze()
    x = outputs.hidden_states[layer].cpu().detach().numpy().squeeze()  

    return x
    # return outputs.last_hidden_state.squeeze(0).detach()  # Embeddings for each token

# Example for a sentence
sentence = dataset['train'][0]['text']  # Replace with the correct column
bert_embeddings = get_embeddings(sentence, tokenizer_bert, model_bert, layer=-1)
gpt2_embeddings = get_embeddings(sentence, tokenizer_gpt, model_gpt, layer=-1)

In [8]:
from scipy.spatial.distance import cosine
import torch

def measure_anisotropy(embeddings):
    # Compute cosine similarities for pairs of embeddings
    cos_similarities = []
    num_samples = 1000  # Adjust for sampling efficiency

    for _ in range(num_samples):
        # print(embeddings.size)
        idx1, idx2 = torch.randint(0, embeddings.shape[0], (2,))
        emb1 = embeddings[idx1]
        # print(emb1)
        sim = 1 - cosine(embeddings[idx1], embeddings[idx2])
        cos_similarities.append(sim)
    
    return sum(cos_similarities) / len(cos_similarities)

bert_anisotropy = measure_anisotropy(bert_embeddings)
gpt2_anisotropy = measure_anisotropy(gpt2_embeddings)
print("BERT Anisotropy:", bert_anisotropy)
print("GPT-2 Anisotropy:", gpt2_anisotropy)

BERT Anisotropy: 0.7162601851025178
GPT-2 Anisotropy: 0.2463051208457339


In [9]:
num_layers_bert = len(model_bert.bert.encoder.layer)
num_layers_bert

12

In [10]:
num_layers_gpt = len(model_gpt.transformer.h)
num_layers_gpt

24

In [11]:
bert_anisotropies = []
for i in range(num_layers_bert):
    bert_embeddings = get_embeddings(sentence, tokenizer_bert, model_bert, layer=i)
    anisotropy = measure_anisotropy(bert_embeddings)
    bert_anisotropies.append(anisotropy)

gpt_anisotropies = []
for i in range(num_layers_gpt):
    gpt_embeddings = get_embeddings(sentence, tokenizer_gpt, model_gpt, layer=i)
    anisotropy = measure_anisotropy(gpt_embeddings)
    gpt_anisotropies.append(anisotropy)

In [12]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

data = {
    'Layer': list(range(num_layers_bert)) + list(range(num_layers_gpt)),
    'Anisotropy': bert_anisotropies + gpt_anisotropies,
    'Model': ['BERT'] * num_layers_bert + ['GPT-2'] * num_layers_gpt
}

df = pd.DataFrame(data)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df[df['Model'] == 'BERT']['Layer'], 
                         y=df[df['Model'] == 'BERT']['Anisotropy'],
                         mode='markers+lines',
                         name='BERT',
                         line=dict(shape='linear', dash='dot')))

fig.add_trace(go.Scatter(x=df[df['Model'] == 'GPT-2']['Layer'], 
                         y=df[df['Model'] == 'GPT-2']['Anisotropy'],
                         mode='markers+lines',
                         name='GPT-2',
                         line=dict(shape='linear', dash='dot')))


fig.update_layout(
    title="Anisotropy Comparison: BERT vs GPT-2",
    xaxis_title="Layer Number",
    yaxis_title="Anisotropy Value",
    legend_title="Model",
)

fig.show()


In [15]:
sentences = dataset['train'][:10]['text']  # Replace with the correct column

In [16]:
joined = " ".join(sentences)

In [17]:
def context_specificity(token, dataset, tokenizer, model, layer=-1):
    embeddings = []
    texts = dataset['text']
    for example in texts:
        try:
            inputs = tokenizer(example, return_tensors="pt")
            outputs = model(**inputs, output_hidden_states=True).hidden_states[layer].squeeze(0).detach()
        except:
            continue

        # Get index of token in the text
        token_id = tokenizer.convert_tokens_to_ids(token)
        if token_id in inputs['input_ids']:
            token_index = (inputs['input_ids'] == token_id).nonzero(as_tuple=True)[1]
            embeddings.append(outputs[token_index].mean(0))  # Averaging over token occurrences

    # Compute average cosine similarity between each pair of embeddings
    cos_similarities = []
    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            sim = 1 - cosine(embeddings[i], embeddings[j])
            cos_similarities.append(sim)

    return sum(cos_similarities) / len(cos_similarities) if cos_similarities else None

# Example usage
bert_context_specificity = context_specificity("nie", dataset['train'][:100], tokenizer_bert, model_bert)
gpt2_context_specificity = context_specificity("nie", dataset['train'][:100], tokenizer_gpt, model_gpt)

print("BERT Context-Specificity:", bert_context_specificity)
print("GPT-2 Context-Specificity:", gpt2_context_specificity)


Token indices sequence length is longer than the specified maximum sequence length for this model (1781 > 512). Running this sequence through the model will result in indexing errors


BERT Context-Specificity: 0.8382445823264426
GPT-2 Context-Specificity: 0.24329707611341647


In [18]:
bert_context = []
for i in range(num_layers_bert):
    context = context_specificity("nie", dataset['train'][:100], tokenizer_bert, model_bert, layer=i)
    bert_context.append(context)

gpt_context = []
for i in range(num_layers_gpt):
    context = context_specificity("nie", dataset['train'][:100], tokenizer_gpt, model_gpt, layer=i)
    gpt_context.append(context)

In [ ]:
data = {
    'Layer': list(range(num_layers_bert)) + list(range(num_layers_gpt)),
    'Context': bert_context + gpt_context,
    'Model': ['BERT'] * num_layers_bert + ['GPT-2'] * num_layers_gpt
}

df = pd.DataFrame(data)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df[df['Model'] == 'BERT']['Layer'], 
                         y=df[df['Model'] == 'BERT']['Context'],
                         mode='markers+lines',
                         name='BERT',
                         line=dict(shape='linear', dash='dot')))

fig.add_trace(go.Scatter(x=df[df['Model'] == 'GPT-2']['Layer'], 
                         y=df[df['Model'] == 'GPT-2']['Context'],
                         mode='markers+lines',
                         name='GPT-2',
                         line=dict(shape='linear', dash='dot')))


fig.update_layout(
    title="Context-Specificity Comparison: BERT vs GPT-2",
    xaxis_title="Layer Index",
    yaxis_title="Context-Specificity Value",
    legend_title="Model",
)

fig.show()

# Parameter Projection


In [14]:
import torch
from torch.nn import functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from tabulate import tabulate
from tqdm import tqdm, trange
from copy import deepcopy
import numpy as np
from collections import Counter

# Helper function - funkcje pomocnicze z notebook'a

In [15]:
ALNUM_CHARSET = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

def convert_to_tokens(indices, tokenizer, extended=False, extra_values_pos=None, strip=True):
    if extended:
        res = [tokenizer.convert_ids_to_tokens([idx])[0] if idx < len(tokenizer) else 
               (f"[pos{idx-len(tokenizer)}]" if idx < extra_values_pos else f"[val{idx-extra_values_pos}]") 
               for idx in indices]
    else:
        res = tokenizer.convert_ids_to_tokens(indices)
    if strip:
        res = list(map(lambda x: x[1:] if x[0] == 'Ġ' else "#" + x, res))
    return res


def top_tokens(v, k=100, tokenizer=None, only_alnum=False, only_ascii=True, with_values=False, 
               exclude_brackets=False, extended=True, extra_values=None, only_from_list=None):
    if tokenizer is None:
        tokenizer = my_tokenizer
    v = deepcopy(v)
    ignored_indices = []
    if only_ascii:
        ignored_indices.extend([key for val, key in tokenizer.vocab.items() if not val.strip('Ġ▁').isascii()])
    if only_alnum: 
        ignored_indices.extend([key for val, key in tokenizer.vocab.items() if not (set(val.strip('Ġ▁[] ')) <= ALNUM_CHARSET)])
    if only_from_list:
        ignored_indices.extend([key for val, key in tokenizer.vocab.items() if val.strip('Ġ▁ ').lower() not in only_from_list])
    if exclude_brackets:
        ignored_indices = set(ignored_indices).intersection(
            {key for val, key in tokenizer.vocab.items() if not (val.isascii() and val.isalnum())})
        ignored_indices = list(ignored_indices)
        
    ignored_indices = list(set(ignored_indices))
    v[ignored_indices] = -np.inf
    extra_values_pos = len(v)
    if extra_values is not None:
        v = torch.cat([v, extra_values])
    values, indices = torch.topk(v, k=k)
    res = convert_to_tokens(indices, tokenizer, extended=extended, extra_values_pos=extra_values_pos)
    if with_values:
        res = list(zip(res, values.cpu().numpy()))
    return res

Extract Weights GPT

In [ ]:
model = AutoModelForCausalLM.from_pretrained("sdadas/polish-gpt2-medium")
tokenizer = my_tokenizer = AutoTokenizer.from_pretrained("sdadas/polish-gpt2-medium")
emb = model.get_output_embeddings().weight.data.T.detach()

GPT2Config {
  "_name_or_path": "sdadas/polish-gpt2-medium",
  "activation_function": "gelu_fast",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": 4096,
  "n_layer": 24,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "tokenizer_class": "GPT2TokenizerFast",
  "torch_dtype": "float32",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 51200
}

In [44]:
emb.shape

torch.Size([1024, 51200])

## Konfugracja modelu

In [45]:
model.config

GPT2Config {
  "_name_or_path": "sdadas/polish-gpt2-medium",
  "activation_function": "gelu_fast",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": 4096,
  "n_layer": 24,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "tokenizer_class": "GPT2TokenizerFast",
  "torch_dtype": "float32",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 51200
}

## Architektura modelu GPT2

In [46]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=False)
)

## Ekstrackja wag z modelu GPT2

In [47]:
num_layers = model.config.n_layer
num_heads = model.config.n_head
hidden_dim = model.config.n_embd
head_size = hidden_dim // num_heads

K = torch.cat(
    [
        model.get_parameter(f"transformer.h.{j}.mlp.c_fc.weight").T
        for j in range(num_layers)
    ]
).detach()
V = torch.cat(
    [
        model.get_parameter(f"transformer.h.{j}.mlp.c_proj.weight")
        for j in range(num_layers)
    ]
).detach()

W_Q, W_K, W_V = (
    torch.cat(
        [
            model.get_parameter(f"transformer.h.{j}.attn.c_attn.weight")
            for j in range(num_layers)
        ]
    )
    .detach()
    .chunk(3, dim=-1)
)
W_O = torch.cat(
    [
        model.get_parameter(f"transformer.h.{j}.attn.c_proj.weight")
        for j in range(num_layers)
    ]
).detach()

K_heads = K.reshape(num_layers, -1, hidden_dim)
V_heads = V.reshape(num_layers, -1, hidden_dim)
d_int = K_heads.shape[1]

W_Q_heads = W_Q.reshape(num_layers, hidden_dim, num_heads, head_size).permute(
    0, 2, 1, 3
)
W_K_heads = W_K.reshape(num_layers, hidden_dim, num_heads, head_size).permute(
    0, 2, 1, 3
)
W_V_heads = W_V.reshape(num_layers, hidden_dim, num_heads, head_size).permute(
    0, 2, 1, 3
)
W_O_heads = W_O.reshape(num_layers, num_heads, head_size, hidden_dim)

emb_inv = emb.T
print("emb_inv", emb_inv.shape)

emb_inv torch.Size([51200, 1024])


## Interpretacja wag modelu GPT2 na pustej liście tokenów

In [48]:
tokens_list = set()

## FF Keys and Values - Klucze i wartości funkcji Feed Forward
- Klucze funkcji feed-forward (FF keys): rzutowane są przez pomnożenie przez transpozycję prawostronnej odwrotności macierzy osadzeń (KE′T)
- Wartości funkcji feed-forward (FF values): rzutowane są przez pomnożenie przez macierz osadzeń E.


In [49]:
print(f"Layers in the model: {num_layers}")
print(f"Neurons in the model: {model.config.n_inner}")

Layers in the model: 24
Neurons in the model: 4096


In [41]:
K_heads.shape

torch.Size([24, 4096, 1024])

In [42]:
V_heads.shape

torch.Size([24, 4096, 1024])

In [33]:
from tabulate import tabulate

def print_top_tokens(i1, i2, K_heads, V_heads, emb, tokens_list=None):

    print(f'Layer {i1}, Neuron {i2}')
    print(
        tabulate(
            [
                *zip(
                    top_tokens(
                        (K_heads[i1, i2]) @ emb,
                        k=30,
                        only_from_list=tokens_list,
                        only_alnum=False,
                    ),
                    top_tokens(
                        (V_heads[i1, i2]) @ emb,
                        k=30,
                        only_from_list=tokens_list,
                        only_alnum=False,
                    ),
                    top_tokens((-K_heads[i1, i2]) @ emb, k=30, only_from_list=tokens_list),
                    top_tokens((-V_heads[i1, i2]) @ emb, k=30, only_from_list=tokens_list),
                )
            ],
            headers=["K", "V", "-K", "-V"],
        )
    )


## layer index and neuron index - indeks warstwy i indeks neuronu

In [ ]:
# Przykład z notebook'a
ilayer = 23
ineuron = 907
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 23, Neuron 907
K          V              -K          -V
---------  -------------  ----------  ---------
przycu     kody           dotychczas  #bot
zalog      #gory          rodzi       #lot
wylegi     #ei            do          #ush
#walifik   #zmy           przebie     #dzista
przesp     Apo            gatunku     #remont
pochowany  apokali        przez       #lee
#cket      #128           #ja         #wan
#ppe       ludy           rodzin      #ette
wep        #ords          zrazu       #up
sfinans    Cezary         lokalnie    #bul
#iss       archa          porywa      #puszczam
#CS        przy           two         spu
#gny       akcy           jeszcze     zamyka
#zwol      Homo           #jaw        odstawi
#-).       litera         na          #mont
lock       #pka           wymaga      #beki
skonfisk   Cezar          ty          #laks
#post      Benedykt       Drze        tap
postoju    narodem        pod         poby
erek       polityki       Nie         posto
#ionu      s

In [37]:
# ostatni neuron w ostatniej warstwie
ilayer = 23
ineuron = 4095
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 23, Neuron 4095
K         V            -K    -V
--------  -----------  ----  ----
#biego    #Olbrzym     d     K
#bieni    #Dzwonek     przy  #y
#arki     #Obywat      z     S
#Byli     #Naraz       i     Hu
#bione    #Organiz     nie   Ko
#biny     #Uczestni    po    Z
#Stacja   #Ostatnim    w     Ch
#basy     #Nikomu      to    Gu
#atki     #Zgoda       od    B
#bice     #Robert      hu    GU
#wory     #Aktor       #,    Ta
#HC       #Weszli      de    Lan
#CJE      #Plat        ku    Mer
#dziny    #Same        do    D
#anami    #Idziemy     sz    G
#hony     #Jakbym      u     o
#bowych   #polsce      l     Sm
#biony    #Jakby       ze    M
#BIE      #Lokal       Hu    #e
#body     #aby         e     U
#bane     #Zdro        mat   Mac
#noscia   #chodzimy    kur   Han
#Pisze    #Coraz       a     #xa
#lektory  #Policj      na    SM
#beli     #Inwestycja  I     AB
#Gmina    #Faj         O     T
#ARE      #Zbiera      pod   O
#Drzewa   #Noc         tego  #ki
#GBT      #Nat       

In [31]:
i1, i2 = 0, 0
print_top_tokens(i1, i2, K_heads, V_heads, emb, tokens_list)

0 0
K        V            -K             -V
-------  -----------  -------------  ---------
#uel     #pus         Tod            #plek
#ul      #ont         #skom          #strzy
#uela    #onek        podzie         #kli
#pul     #sud         #dzie          #wat
#gul     #onych       #sce           #liwie
#ann     #TK          #wie           #rgi
bel      #STWO        #max           uszy
#jal     #oper        #zenia         #RI
#cht     #oty         #stycznych     #las
#oll     #BW          rezerwat       #gre
#El      #ARS         predy          podze
#ulf     #poty        #system        #stralij
El       #ones        niespodzianki  #dle
el       postoju      niespodzianka  #nisz
dywiden  #det         #styczne       #dney
#alu     #sbur        #kaza          #genera
#atak    konspiracji  #rom           #inga
#itt     #ono         ciekawostki    rozdz
#chol    #pes         #rick          usposobie
#anga    #rzac        #dac           #rwi
#al      ognisk       przygod        #rodzi
#ema

Covid i kraje

In [53]:
ilayer = 2
ineuron = 1654
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 2, Neuron 1654
K            V           -K        -V
-----------  ----------  --------  ---------
#soci        Tob         #czynie   stopnia
immuno       habili      #liny     #pty
Demokracji   #mund       #iali     #day
newslet      Bag         #ujecie   sani
#walony      Sztokhol    #Ul       konserw
byle         Kopenha     #linie    #ore
emerytalne   #isy        #czyny    #anci
COVID        operacyjny  #olan     #ptom
rektora      Leningra    Bell      #antka
podro        norwe       #line     parki
glowa        #lda        #57       spos
covid        Sopo        #id       #peratura
#rek         #targ       #bar      #leja
Trybun       #sio        #tonu     #lee
#ytany       #szen       #lic      #post
Europejskim  Wolf        #ian      #walent
Uniwersyte   kame        #osenki   #pcie
panstwa      szwe        #52       #stadt
#Nikogo      autorskie   #gramu    #oro
ryja         Bere        #belki    #lecz
#misja       ptak        #ai       #pania
Niego        #szeniach   #pal

Przestępstwa? kryminalistyka?

In [67]:
ilayer = 22
ineuron = 3360
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 22, Neuron 3360
K            V       -K         -V
-----------  ------  ---------  -----------
narkoty      #tos    rozmie     #redy
#zwykopem    #TM     #aku       #oner
#boli        #czek   G          #tyw
alkoholizm   #128    #d         #illi
#eksyka      #dnik   Fryd       #nera
halu         #Http   Kl         #Kultura
przedaw      #rowy   #th        #syw
porwania     #razem  #harmo     emigracji
#ect         szczeg  M          kampanii
narko        #miast  wojewody   autorskich
pej          Byto    rozlo      demokra
narkotyki    #gua    dobud      #pij
#nezyj       Get     Dwo        #iry
#bolu        ogniem  poddanych  urlopu
#Poszuki     #czke   Lit        #UN
halucyna     #ciu    Th         #Kandy
#tnamie      #raz    #ospo      #zy
narkotykami  Stal    K          #bur
afera        #razu   Gale       referen
#tnam        rzek    #St        #tywa
dopala       #dalem  #polska    #iller
seksualne    Sad     Pa         #ersona
gangster     #dniki  wiernie    celu
temat      

Teorie spiskowe

In [69]:
ilayer = 12
ineuron = 3763
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 12, Neuron 3763
K           V           -K          -V
----------  ----------  ----------  -------------
porwania    Smole       wachlarz    #ymon
FBI         #rzany      staw        #pen
samolotu    smole       #nop        #amu
#rory       #rory       #styki      #kiel
#ash        UFO         Cuk         #eno
NSA         #rany       #mus        akompa
zestrze     Marsa       #klasy      niskie
podkomisji  wahad       wstaw       niskich
lotu        Czecze      #styka      administracji
afery       #had        oliw        drewno
#hera       wirusa      stawu       drew
#Charlie    #ralni      wydziela    odczu
#ond        abor        #lator      #chol
terrorysty  #rzani      #stw        #eny
inwi        sekty       korzeni     #ymu
#154        porwania    wzniesi     #fonii
#lotem      WSI         wargach     #standar
rejs        Katy        #mienia     monoton
#ulf        #TA         #szczyzna   odzwierciedla
CIA         #dliwie     #kolor      desek
samolocie   #biec       ()  

Afganistan

In [80]:
ilayer = 5
ineuron = 2031
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 5, Neuron 2031
K               V          -K           -V
--------------  ---------  -----------  -----------
Afganistanu     Rzesz      #reb         emerytalne
Afga            #parcie    fair         publiczne
Jehowy          #tyz       #gle         #szto
Wojny           #stie      #cie         ui
Obywatelskich   #tz        #bal         #smo
zbiorowych      #kop       #patrz       Sub
etni            #tkami     #arter       #szard
getta           #zdro      #gl          publiczna
rdzenia         #lock      moimi        odprawy
#niegdzie       #tto       mym          powsze
Kinga           #TS        #bki         #Ur
etnicznych      #wcze      moim         autorskie
Afganistanie    #ieu       #pi          Anne
wojnach         uderzeniu  #de          #mion
#zacje          rzesz      #lar         integra
przegranej      #sci       pas          ur
emerytalnych    #zej       biurko       publicznych
wojennych       #zno       #icho        celne
Krajowych       hamowania  #dgo        

nazwy imiona

In [94]:
ilayer = 19
ineuron = 0
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 19, Neuron 0
K            V              -K      -V
-----------  -------------  ------  --------
imiona       pseudonim      #a      #uszka
imion        nazwy          so      #tari
nazw         #naz           #to     #usze
nazwa        nazw           na      #strzy
pseudonim    krypto         w       Kore
nazwy        przydomek      #,      #lander
przydomek    imienia        #lo     Cmenta
nazwisko     nazwa          #li     #rzymie
#Nazwa       nazew          i       #taria
imienia      imion          rewi    #bara
nazwisk      imiona         al      czem
pseudonimem  oznaczenie     #c      Serce
oznaczenie   nazwisko       b       #aga
imieniem     pseudonimem    #lota   Gier
termino      kod            #letni  #wi
nazwiskiem   specjalizacji  do      #uszek
nazew        #Nazwa         #kor    #ariuszy
nazwiska     przydom        ra      jo
nazywam      #okre          bram    ustrze
nazwi        termino        #k      #ymber
nazwali      nume           wy      Atenach
nazwano 

Ameryka

In [95]:
ilayer = 19
ineuron = 1035
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 19, Neuron 1035
K             V         -K        -V
------------  --------  --------  ----------
#Trump        Aut       #sz       #yj
#Bill         #daci     #beli     #towa
Jackson       #niach    Cza       #ansu
American      die       #wieni    #ull
Waszyngtonie  #VD       #wicz     #turem
Josh          bio       Strze     #ty
nowojor       #dym      Ko        Szczecina
#Ameryka      #die      #dowska   #tyna
Kennedy       #dia      #rgi      #sik
Carter        #niczymi  mundu     Szczecinie
#Produkty     #dzeni    strze     #jonu
#Will         #zdni     Bere      #szek
Warren        #ica      #wi       #rum
Trump         Abdu      #nt       #cjum
#neya         ulot      patro     Szczepa
Kalifor       #niczym   najpierw  #siki
#John         Hom       Gie       #ulu
Vegas         lek       gi        #szczek
#hire         #dzie     Bu        #eze
Wash          #ninie    Woj       #ucha
Angeles       #de       dach      #szt
#okracji      #twem     #r        #Lata
Davis       

Miejsca

In [107]:
ilayer = 12
ineuron = 970
print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 12, Neuron 970
K             V           -K          -V
------------  ----------  ----------  -------------
Kolumbii      Spring      #lnik       #zes
polonij       Chal        przeciwnik  #bour
Estonii       Kosza       #bek        #owatych
Alba          Spa         #czaj       #szcie
aero          Pompej      #dziec      #rzego
alpej         Nort        #tch        polityczny
Lubli         Algier      #lna        #atyw
spalin        Chrz        #lt         dyskryminacji
Katowicach    Cez         #za         wystawy
#hausen       Gdynia      #l          rekomendacji
epizo         #nton       #lowi       turystyczny
berli         #nit        #gon        #anki
Esto          Sur         #chta       poselski
gospodarstwa  #nian       #Jego       opublikowany
Katowice      #nity       #zyc        artystyczny
Sano          Ari         #kusa       ustawowych
NRD           #nowiec     #zy         odrzucenia
#derni        Stanach     #ego        #towanych
benzy         #uston      #resz 

In [106]:
ilayer, ineuron = np.random.randint(num_layers), np.random.randint(d_int)

print_top_tokens(ilayer, ineuron, K_heads, V_heads, emb, tokens_list)

Layer 12, Neuron 970
K             V           -K          -V
------------  ----------  ----------  -------------
Kolumbii      Spring      #lnik       #zes
polonij       Chal        przeciwnik  #bour
Estonii       Kosza       #bek        #owatych
Alba          Spa         #czaj       #szcie
aero          Pompej      #dziec      #rzego
alpej         Nort        #tch        polityczny
Lubli         Algier      #lna        #atyw
spalin        Chrz        #lt         dyskryminacji
Katowicach    Cez         #za         wystawy
#hausen       Gdynia      #l          rekomendacji
epizo         #nton       #lowi       turystyczny
berli         #nit        #gon        #anki
Esto          Sur         #chta       poselski
gospodarstwa  #nian       #Jego       opublikowany
Katowice      #nity       #zyc        artystyczny
Sano          Ari         #kusa       ustawowych
NRD           #nowiec     #zy         odrzucenia
#derni        Stanach     #ego        #towanych
benzy         #uston      #resz 

# Attention Weights interpretation - Interpretacja wag modulu atencji

Rzutując parametry do przestrzeni osadzeń, można je interpretować w kategoriach interakcji między słowami ze słownika. Na przykład:
- W przypadku macierzy WQK można zidentyfikować pary słów, które silnie ze sobą współgrają w kontekście mechanizmu uwagi.
- W przypadku macierzy WVO można zobaczyć, jak uwaga na pewne słowa wpływa na kolejne ukryte stany modelu, a co za tym idzie, na ostateczne przewidywania

disclaimer:

W praktyce autorzy używają transpozycji macierzy osadzeń (ET) jako prawostronnej odwrotności E, zamiast bardziej odpowiedniej teoretycznie pseudo-odwrotności (E+). Dzieje się tak, ponieważ interpretacja w przestrzeni osadzeń często polega na wybieraniu k najważniejszych elementów (top-k), a pseudo-odwrotność nie zachowuje się dobrze w połączeniu z tą operacją.

In [135]:
def approx_topk(mat, min_k=500, max_k=250_000, th0=10, max_iters=10, verbose=False):
    _get_actual_k = lambda th, th_max: torch.nonzero((mat > th) & (mat < th_max)).shape[
        0
    ]
    th_max = np.inf
    left, right = 0, th0
    while True:
        actual_k = _get_actual_k(right, th_max)
        if verbose:
            print(f"one more iteration. {actual_k}")
        if actual_k <= max_k:
            break
        left, right = right, right * 2
    if min_k <= actual_k <= max_k:
        th = right
    else:
        for _ in range(max_iters):
            mid = (left + right) / 2
            actual_k = _get_actual_k(mid, th_max)
            if verbose:
                print(f"one more iteration. {actual_k}")
            if min_k <= actual_k <= max_k:
                break
            if actual_k > max_k:
                left = mid
            else:
                right = mid
        th = mid
    return torch.nonzero((mat > th) & (mat < th_max)).tolist()


def get_top_entries(
    tmp,
    all_high_pos,
    only_ascii=False,
    only_alnum=False,
    exclude_same=False,
    exclude_fuzzy=False,
    tokens_list=None,
    reverse_list=False,
):
    remaining_pos = all_high_pos
    if only_ascii:
        remaining_pos = [
            *filter(
                lambda x: (
                    tokenizer.decode(x[0]).strip("Ġ▁").isascii()
                    and tokenizer.decode(x[1]).strip("Ġ▁").isascii()
                ),
                remaining_pos,
            )
        ]
    if only_alnum:
        remaining_pos = [
            *filter(
                lambda x: (
                    tokenizer.decode(x[0]).strip("Ġ▁ ").isalnum()
                    and tokenizer.decode(x[1]).strip("Ġ▁ ").isalnum()
                ),
                remaining_pos,
            )
        ]
    if exclude_same:
        remaining_pos = [
            *filter(
                lambda x: tokenizer.decode(x[0]).lower().strip()
                != tokenizer.decode(x[1]).lower().strip(),
                remaining_pos,
            )
        ]
    if exclude_fuzzy:
        remaining_pos = [
            *filter(
                lambda x: not _fuzzy_eq(
                    tokenizer.decode(x[0]).lower().strip(),
                    tokenizer.decode(x[1]).lower().strip(),
                ),
                remaining_pos,
            )
        ]
    if tokens_list:
        remaining_pos = [
            *filter(
                lambda x: (
                    (tokenizer.decode(x[0]).strip("Ġ▁").lower().strip() in tokens_list)
                    and (
                        tokenizer.decode(x[1]).strip("Ġ▁").lower().strip()
                        in tokens_list
                    )
                ),
                remaining_pos,
            )
        ]

    pos_val = tmp[[*zip(*remaining_pos)]]
    good_cells = [
        *map(lambda x: (tokenizer.decode(x[0]), tokenizer.decode(x[1])), remaining_pos)
    ]
    good_tokens = list(map(lambda x: Counter(x).most_common(), zip(*good_cells)))
    remaining_pos_best = np.array(remaining_pos)[
        torch.argsort(pos_val if reverse_list else -pos_val)[:50]
    ]
    good_cells_best = [
        *map(
            lambda x: (tokenizer.decode(x[0]), tokenizer.decode(x[1])),
            remaining_pos_best,
        )
    ]
    # good_cells[:100]
    # list(zip(good_tokens[0], good_tokens[1]))
    return good_cells_best

## WVO Interpretation
Interpretacja Wvo polega na analizie tej macierzy przejścia, identyfikując pary słów, które są silnie ze sobą powiązane.

In [136]:
def analyze_weights_vo(i1, i2, exclude_same=False, reverse_list=False, only_ascii=True, only_alnum=False, tokens_list=None):
    print(f"Layer {i1}, Head {i2}")

    # Extract the weights for the specified layer and neuron
    W_V_tmp, W_O_tmp = W_V_heads[i1, i2, :], W_O_heads[i1, i2]
    
    # Compute the temporary matrix using emb_inv
    tmp = emb_inv @ (W_V_tmp @ W_O_tmp) @ emb
    
    # Approximate top-k values
    all_high_pos = approx_topk(tmp, th0=1, verbose=True)
    
    # Get top entries based on the specified parameters
    top_entries = get_top_entries(
        tmp,
        all_high_pos,
        only_ascii=only_ascii,
        only_alnum=only_alnum,
        exclude_same=exclude_same,
        tokens_list=tokens_list,
        reverse_list=reverse_list
    )
    
    return top_entries


Przedrostki

In [ ]:
# Example usage
ilayer, head = 21, 7
analyze_weights_vo(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=False,
    tokens_list=None,
)

one more iteration. 0
one more iteration. 0
one more iteration. 10
one more iteration. 5182


[(' ranem', ' nad'),
 ('ornie', ' przez'),
 ('datek', ' nad'),
 ('arl', ' przed'),
 (' pewno', ' na'),
 ('spodziewanie', ' nad'),
 (' razu', ' od'),
 ('miernie', ' nad'),
 (' okazji', ' przy'),
 (' wsk', ' na'),
 (' czele', ' na'),
 ('orne', ' przez'),
 ('godziny', ' nad'),
 ('sione', ' przed'),
 (' ranem', 'nad'),
 ('natural', ' nad'),
 ('przewo', ' nad'),
 (' Duna', ' nad'),
 ('ktory', ' do'),
 ('miar', ' nad'),
 (' dobra', ' dla'),
 (' Jezi', ' nad'),
 ('spodzie', ' nad'),
 (' niedawna', ' do'),
 ('pisie', ' pod'),
 (' wygody', ' dla'),
 ('arcie', ' przed'),
 (' sumie', ' w'),
 ('miernie', 'nad'),
 ('tek', ' pod'),
 ('wcze', ' przed'),
 ('mier', ' nad'),
 ('hala', ' pod'),
 ('ornie', ' przeze'),
 (' uboczu', ' na'),
 (' podstawie', ' na'),
 (' ranem', 'Nad'),
 ('czesne', ' do'),
 ('granicznych', ' nad'),
 ('wiska', ' przez'),
 (' barkach', ' na'),
 ('ornie', 'przez'),
 (' odmiany', ' dla'),
 ('niego', ' przed'),
 (' plecami', ' za'),
 (' dobi', ' na'),
 ('przewodni', ' nad'),
 (' ko

swoich swoje siebie

In [138]:
ilayer, head = 11, 12
analyze_weights_vo(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=False,
    tokens_list=None,
)

Layer 11, Head 12
one more iteration. 0
one more iteration. 0
one more iteration. 0
one more iteration. 13
one more iteration. 15010


[(' komukolwiek', ' komukolwiek'),
 ('siebie', 'siebie'),
 (' swoich', ' swoich'),
 (' swojej', ' swojej'),
 (' swej', ' swojej'),
 ('scie', 'scie'),
 (' swojej', ' swoich'),
 ('ingu', 'ingu'),
 (' swoje', ' swoich'),
 (' swoich', ' swojej'),
 (' swoja', ' swoich'),
 (' swych', ' swoich'),
 (' swoja', ' swojej'),
 (' siebie', ' siebie'),
 (' swoja', ' swoim'),
 (' swej', ' swej'),
 (' swoim', ' swoim'),
 (' swych', ' swojej'),
 (' bezkarnie', ' bezkarnie'),
 ('siebie', ' siebie'),
 ('rko', 'rko'),
 (' swych', ' swych'),
 (' swoje', ' swoje'),
 ('imon', 'imon'),
 (' swoich', ' swych'),
 (' siebie', 'siebie'),
 (' swej', ' swoich'),
 ('bes', 'bes'),
 (' sobie', ' sobie'),
 (' swej', ' swych'),
 (' swoje', ' swojej'),
 ('dun', 'dun'),
 (' swojej', ' swych'),
 (' swoim', ' swojej'),
 (' swojej', ' swoim'),
 (' swojej', ' swej'),
 (' swoje', ' swoim'),
 (' swoim', ' swoich'),
 (' swojego', ' swoich'),
 (' swoja', ' swych'),
 ('ujecie', 'ujecie'),
 (' beze', ' beze'),
 (' swoich', ' swojego'

tobie, ciebie, twoimi

In [142]:
ilayer, head = 20, 13
analyze_weights_vo(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=False,
    tokens_list=None,
)

Layer 20, Head 13
one more iteration. 358
one more iteration. 11868


[(' jej', ' jej'),
 (' jej', ' ona'),
 (' Ciebie', ' Ciebie'),
 (' jej', ' Ona'),
 (' wjej', ' jej'),
 (' Ciebie', ' Tobie'),
 (' ciebie', ' ciebie'),
 (' Twoich', ' Ciebie'),
 (' ich', ' ich'),
 (' ciebie', ' tobie'),
 (' Ciebie', ' Twojej'),
 (' Twojej', ' Ciebie'),
 (' Ciebie', ' Twoich'),
 (' twoimi', ' ciebie'),
 (' wjej', ' ona'),
 (' Twoim', ' Ciebie'),
 (' Ciebie', ' Twoim'),
 (' twoich', ' ciebie'),
 (' Twoje', ' Ciebie'),
 (' twoje', ' ciebie'),
 (' jej', ' wjej'),
 (' Ciebie', ' Twoje'),
 (' Twoich', ' Tobie'),
 (' wjej', ' Ona'),
 (' Tobie', ' Ciebie'),
 (' Twoich', ' Twoich'),
 (' ciebie', ' twoich'),
 (' Twoich', ' Twojej'),
 (' Ci', ' Ciebie'),
 (' ciebie', ' twoimi'),
 (' twoim', ' ciebie'),
 (' Twoim', ' Twoich'),
 (' ciebie', ' twojej'),
 (' was', ' was'),
 (' ciebie', ' twoim'),
 (' twoimi', ' tobie'),
 (' jej', ' niej'),
 (' jej', ' Jej'),
 (' was', ' wami'),
 (' twoich', ' tobie'),
 (' tobie', ' ciebie'),
 (' Twoich', ' Twoje'),
 (' twoich', ' twoich'),
 (' Twojej'

spójniki opisujące położenie

In [147]:
ilayer, head = 19, 7
analyze_weights_vo(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=False,
    tokens_list=None,
)

Layer 19, Head 7
one more iteration. 0
one more iteration. 0
one more iteration. 374
one more iteration. 34965


[(' czele', ' na'),
 (' pryzmat', ' przez'),
 (' froncie', ' na'),
 (' etapie', ' na'),
 (' pewno', ' na'),
 ('miast', ' na'),
 (' powodu', ' z'),
 (' ekranie', ' na'),
 (' pryzmat', 'przez'),
 ('rzuca', ' do'),
 (' posiedzeniu', ' na'),
 (' tarasie', ' na'),
 ('ping', ' do'),
 ('ornie', ' przez'),
 (' smyczy', ' na'),
 (' poziomie', ' na'),
 ('ornie', ' przeze'),
 (' razu', ' od'),
 (' Wyspach', ' na'),
 (' dawna', ' od'),
 ('ted', ' w'),
 (' perspektywie', ' w'),
 (' terenach', ' na'),
 (' momencie', ' w'),
 ('orne', ' przez'),
 (' francusku', ' po'),
 (' ziemiach', ' na'),
 (' rezultacie', ' w'),
 (' wymiarze', ' w'),
 (' Twitterze', ' na'),
 (' barkach', ' na'),
 ('gnieniu', ' w'),
 ('leg', ' na'),
 (' mgnieniu', ' we'),
 (' posiedzeniach', ' na'),
 (' kresu', ' u'),
 (' mgnieniu', ' w'),
 (' razie', ' na'),
 (' rana', ' od'),
 (' pryzmat', ' przeze'),
 ('tedy', ' we'),
 ('ted', ' we'),
 (' podstawie', ' na'),
 (' sensie', ' w'),
 ('ornie', 'przez'),
 (' niemiecku', ' po'),
 (' ryn

spójniki inaczej pisane

In [154]:
ilayer, head = 19, 1
analyze_weights_vo(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=False,
    tokens_list=None,
)

Layer 19, Head 1
one more iteration. 0
one more iteration. 2
one more iteration. 639


[(' przed', ' przed'),
 (' nad', ' nad'),
 (' miedzy', ' miedzy'),
 ('tylko', 'tylko'),
 (' przed', 'przed'),
 ('poza', 'poza'),
 (' poza', ' poza'),
 (' nad', 'nad'),
 (' sam', ' sam'),
 (' pod', ' pod'),
 ('Nad', ' nad'),
 (' osobna', ' osobna'),
 ('przed', ' przed'),
 (' nad', 'Nad'),
 (' osobno', ' osobno'),
 ('poza', ' poza'),
 (' dalej', ' dalej'),
 ('tylko', ' tylko'),
 (' sama', ' sam'),
 (' poza', 'poza'),
 ('tylko', ' jeno'),
 (' same', ' same'),
 (' sam', ' sama'),
 (' najmocniej', ' najmocniej'),
 (' przed', 'Przed'),
 (' osobno', ' oddzielnie'),
 (' sama', ' sama'),
 (' oddzielnie', ' osobno'),
 ('Nad', 'nad'),
 (' same', ' sam'),
 (' najszybciej', ' najszybciej'),
 ('Nad', 'Nad'),
 (' dopiero', ' dopiero'),
 (' osobno', ' osobna'),
 (' same', ' sama'),
 ('przed', 'przed'),
 (' oddzielnie', ' oddzielnie'),
 (' osobna', ' osobno'),
 (' najdalej', ' najdalej'),
 (' kto', ' kto'),
 (' nade', ' nad'),
 (' razem', ' razem'),
 (' osobna', ' oddzielnie'),
 ('tylko', ' jedynie'),


inne...

In [172]:
ilayer, head = np.random.randint(num_layers), np.random.randint(num_heads)
analyze_weights_vo(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=False,
    tokens_list=None,
)

Layer 21, Head 4
one more iteration. 0
one more iteration. 0
one more iteration. 5
one more iteration. 17496


[('zjaty', ' A'),
 ('rycznie', ' Li'),
 (' akcyz', ' opodat'),
 (' podatku', ' podatek'),
 ('onel', ' Li'),
 ('steri', ' M'),
 (' podatkach', ' podatki'),
 ('portu', ' Ra'),
 (' podatku', ' podatku'),
 ('owisk', ' R'),
 (' podatkach', ' podatkach'),
 (' podatkach', ' podatek'),
 ('nonim', ' A'),
 ('garnia', ' O'),
 ('zon', ' D'),
 ('mami', ' O'),
 ('gresyw', ' A'),
 (' akcyz', ' opodatkowania'),
 (' koszt', ' koszt'),
 ('aryka', ' K'),
 ('tery', ' Li'),
 ('boli', ' A'),
 ('zmat', ' G'),
 (' Saudyjskiej', ' Li'),
 ('niowym', ' Li'),
 ('warantan', ' K'),
 (' podatkach', ' podatku'),
 (' podatku', ' podatki'),
 ('aliz', ' R'),
 (' akcyz', ' podatek'),
 ('wagi', ' U'),
 ('ure', ' Ma'),
 ('pelu', ' Ka'),
 ('niowej', ' Li'),
 ('gazy', ' Ma'),
 ('rygi', ' O'),
 ('ryczny', ' Li'),
 (' akcyz', ' podatkowych'),
 (' podatkiem', ' opodat'),
 (' podatku', ' podatkowym'),
 ('seum', ' Li'),
 ('et', ' R'),
 (' koszty', ' koszty'),
 (' akcyz', ' podatkowe'),
 (' akcyz', ' podatku'),
 (' podatek', ' pod

## WQK Interpretation
Interpretacja Wqk polega na analizie tej macierzy, identyfikując pary słów, które silnie ze sobą współgrają w kontekście mechanizmu uwagi

In [118]:
def analyze_weigths_qk(i1, i2, exclude_same=False, reverse_list=False, only_ascii=True, only_alnum=True, tokens_list=None):
    print(f"Layer {i1}, Head {i2}")

    # Extract the weights for the specified layer and neuron
    W_Q_tmp, W_K_tmp = W_Q_heads[i1, i2, :], W_K_heads[i1, i2, :]

    # Compute the temporary matrix using emb_inv and emb_inv.T
    tmp = emb_inv @ (W_Q_tmp @ W_K_tmp.T) @ emb_inv.T

    # Approximate top-k values
    all_high_pos = approx_topk(tmp, th0=1, verbose=True)

    # Get top entries based on the specified parameters
    top_entries = get_top_entries(
        tmp,
        all_high_pos,
        only_ascii=only_ascii,
        only_alnum=only_alnum,
        exclude_same=exclude_same,
        tokens_list=tokens_list,
        reverse_list=reverse_list
    )

    return top_entries

In [119]:
layer, head = 21, 7
analyze_weigths_qk(
    layer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=True,
    tokens_list=None,
)

Layer 21, Head 7
one more iteration. 0
one more iteration. 0
one more iteration. 42
one more iteration. 316744
one more iteration. 4556


[(' za', 'noscia'),
 ('dnieniem', ' przy'),
 (' pod', 'Uwiel'),
 (' w', 'noscia'),
 (' w', 'kowicz'),
 ('aryjnych', ' tylko'),
 ('aryjnych', ' po'),
 (' w', 'knal'),
 (' z', 'noscia'),
 (' w', 'dagogi'),
 (' przy', 'owienia'),
 (' i', 'noscia'),
 (' pod', 'cjonista'),
 (' pod', 'nujesz'),
 (' nad', 'ngu'),
 (' przy', 'Pry'),
 ('jutrz', ' w'),
 (' pod', 'alski'),
 (' w', 'lujesz'),
 (' przy', ' budowlane'),
 ('aryjnych', ' przy'),
 (' na', 'noscia'),
 ('aryjnych', ' nawet'),
 ('Przytak', ' przy'),
 (' na', 'owepaski'),
 (' po', 'bil'),
 (' od', 'dagogi'),
 (' w', 'URWIA'),
 (' pod', ' glowa'),
 (' po', 'sywnie'),
 (' do', 'wychw'),
 ('aryjnych', ' zak'),
 (' nad', 'obior'),
 (' pod', 'niuje'),
 (' pod', 'zc'),
 (' w', 'towaniem'),
 (' pod', 'Przytu'),
 (' przed', 'ingiem'),
 (' o', 'bino'),
 (' w', 'tujesz'),
 (' do', 'ariatu'),
 (' dla', ' pryzmat'),
 (' pod', ' Wojskowego'),
 (' po', 'dujesz'),
 (' i', 'cjonista'),
 (' z', 'kowicz'),
 (' na', 'dagogi'),
 (' w', 'ustu'),
 (' o', 'nosci

Podróże linie lotnicze

In [123]:
ilayer, head = 0, 5 
analyze_weigths_qk(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=True,
    tokens_list=None,
)

Layer 0, Head 5
one more iteration. 0
one more iteration. 0
one more iteration. 14860


[(' konsulat', ' rzymski'),
 (' interpretacje', ' podatkowych'),
 (' liniach', ' lotnicze'),
 (' pokoju', ' hotel'),
 (' linii', ' tramwaju'),
 (' liniach', ' lotniczy'),
 (' liniach', ' lotniczych'),
 (' kurator', ' sztuki'),
 (' liniach', ' tramwaju'),
 (' terminalu', ' lotni'),
 (' kuratora', ' sztuki'),
 (' konsulat', ' Rzymi'),
 (' terminalu', ' lotniczych'),
 (' interpretacji', ' podatkowych'),
 (' interpretacje', ' podatkowego'),
 (' linie', ' lotnicze'),
 (' liniach', ' lotniczego'),
 (' liniach', ' tramwaj'),
 (' liniach', ' samolo'),
 (' liniach', ' tramwaje'),
 (' liniach', ' tramwa'),
 (' liniach', ' samolocie'),
 (' liniach', ' kolejowa'),
 (' konsulat', ' Cezara'),
 (' linii', ' kolejowa'),
 (' linie', ' lotniczej'),
 (' linie', ' lotniczy'),
 (' linie', ' lotniczych'),
 (' linie', ' samolo'),
 (' interpretacje', ' podatkowe'),
 (' linie', ' kolejowe'),
 (' linii', 'Samolot'),
 (' terminalu', ' lotnicze'),
 (' linie', 'Samolot'),
 (' linii', ' lotniczych'),
 (' interpreta

pan, panem, pani

In [127]:
ilayer, head = 23, 15
analyze_weigths_qk(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=True,
    tokens_list=None,
)

Layer 23, Head 15
one more iteration. 0
one more iteration. 573


[(' panem', ' pan'),
 (' pana', ' pan'),
 (' pan', ' panu'),
 (' pana', ' panu'),
 (' panem', ' panu'),
 (' pan', ' pan'),
 (' panem', ' pana'),
 (' pana', ' pana'),
 (' pan', ' pana'),
 (' panu', ' panu'),
 (' Pani', ' Pani'),
 ('Pana', ' pan'),
 (' pana', ' Panu'),
 (' pan', ' Panu'),
 (' panu', ' pan'),
 ('panem', ' pan'),
 (' Panem', ' pan'),
 (' pana', ' Pan'),
 (' pani', ' Pani'),
 (' pana', 'pan'),
 (' pana', ' panowie'),
 (' panem', ' Pan'),
 (' pan', 'pan'),
 (' pan', ' panem'),
 (' panu', ' pana'),
 (' pan', ' Pan'),
 ('Pani', ' pani'),
 (' pana', ' Pana'),
 ('Pani', ' Pani'),
 ('panem', ' panu'),
 (' pan', ' Pana'),
 (' panem', ' Panu'),
 (' pana', ' panem'),
 (' pan', ' panom'),
 ('Pana', ' pana'),
 ('Pana', ' panu'),
 (' Pana', ' pan'),
 ('Pan', ' pan'),
 (' panom', ' pan'),
 ('Pan', ' pana'),
 (' Panem', ' panu'),
 (' Panem', ' pana'),
 (' panem', ' Pana'),
 (' pani', ' pani'),
 (' pana', ' panom'),
 (' panem', ' panem'),
 ('panem', ' pana'),
 ('Pan', ' panu'),
 (' panu',

formy grzecznościowe?

In [130]:
ilayer, head = 8, 1
analyze_weigths_qk(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=True,
    tokens_list=None,
)

Layer 8, Head 1
one more iteration. 0
one more iteration. 0
one more iteration. 67
one more iteration. 39053


[(' gdybym', ' gdybym'),
 (' zapraszam', ' jakbym'),
 (' moimi', ' jakbym'),
 (' gdybym', ' jakbym'),
 (' zapraszam', ' mojej'),
 (' abym', ' jakbym'),
 (' jakbym', ' jakbym'),
 (' nacjona', 'ownictwo'),
 ('erytory', ' percep'),
 (' moich', ' jakbym'),
 ('erytory', 'zumiem'),
 (' abym', 'lewam'),
 (' zapraszam', ' umiem'),
 ('interpre', 'alizmu'),
 (' zapraszam', ' moja'),
 (' moimi', ' moich'),
 (' czytasz', 'rod'),
 (' zapraszam', ' czekam'),
 (' prosimy', ' znajdziecie'),
 (' jakbym', ' gdybym'),
 (' moimi', ' gdybym'),
 (' skutkiem', ' prania'),
 (' abym', ' abym'),
 (' abym', ' gdybym'),
 (' kryzysu', ' wojennego'),
 (' zapraszam', ' gdybym'),
 (' jakbym', 'lewam'),
 (' zapraszam', ' moich'),
 (' mylisz', ' waszej'),
 ('erytory', ' wieczystych'),
 (' jakbym', ' umiem'),
 ('erytory', 'rach'),
 (' moimi', ' abym'),
 (' prosze', ' jakbym'),
 ('sen', ' tvn'),
 (' uszczerbku', ' prania'),
 (' zapraszam', ' szukam'),
 (' moimi', 'lewam'),
 (' wasza', ' waszych'),
 ('gnu', 'rod'),
 (' mo

Spójniki

In [133]:
ilayer, head = 17, 15
analyze_weigths_qk(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=True,
    tokens_list=None,
)

Layer 17, Head 15
one more iteration. 0
one more iteration. 0
one more iteration. 46
one more iteration. 110979


[(' itp', ' etc'),
 (' itp', ' itp'),
 (' itp', ' itd'),
 (' etc', ' etc'),
 (' itd', ' itd'),
 (' itd', ' etc'),
 (' ktora', ' jesli'),
 (' ktore', ' jesli'),
 (' etc', ' itp'),
 (' itd', ' itp'),
 (' etc', ' itd'),
 (' ktorzy', ' jesli'),
 (' jezeli', ' jesli'),
 (' ktore', ' jezeli'),
 (' gdyz', ' jesli'),
 (' ktora', ' jezeli'),
 (' jezeli', ' jezeli'),
 (' ktory', ' jesli'),
 (' poniewaz', ' jesli'),
 (' ktorym', ' jesli'),
 (' gdyz', ' poniewaz'),
 (' gdyz', ' gdyz'),
 (' ktory', ' jezeli'),
 (' ktorych', ' jesli'),
 (' ktora', ' choc'),
 (' gdyz', ' choc'),
 (' ktore', ' choc'),
 (' jesli', ' jesli'),
 (' ktorzy', ' jezeli'),
 (' gdyz', ' gdy'),
 (' zeby', ' jesli'),
 (' poniewaz', ' poniewaz'),
 (' ktorym', ' jezeli'),
 (' ktor', ' jesli'),
 ('plementa', ' i'),
 (' ktor', ' jezeli'),
 (' troche', ' ktorych'),
 (' poniewaz', ' jezeli'),
 (' wiecej', ' ktorych'),
 (' zeby', ' jezeli'),
 (' kogos', ' ktorych'),
 (' jakiegos', ' zeby'),
 (' ktorego', ' jesli'),
 (' jesli', ' jezeli

In [134]:
ilayer, head = np.random.randint(num_layers), np.random.randint(num_heads)
analyze_weigths_qk(
    ilayer,
    head,
    exclude_same=False,
    reverse_list=False,
    only_ascii=True,
    only_alnum=True,
    tokens_list=None,
)

Layer 0, Head 12
one more iteration. 0
one more iteration. 262
one more iteration. 27303


[('wala', 'la'),
 ('walam', 'la'),
 (' City', ' Schengen'),
 (' stacji', ' Schengen'),
 ('lam', 'la'),
 (' aplikacji', ' Schengen'),
 (' ja', 'la'),
 (' przejazdu', ' Schengen'),
 (' lokalizacji', ' socjologii'),
 (' trasy', ' turystycznej'),
 (' lokalizacji', ' policj'),
 (' lokalizacji', ' Schengen'),
 (' stacji', ' policj'),
 (' akcji', ' turystycznej'),
 (' inwestycji', ' Schengen'),
 ('wal', 'la'),
 (' kolonii', ' Schengen'),
 (' terenu', ' policj'),
 (' lokalizacji', ' turystyczny'),
 (' lokalizacji', ' depor'),
 ('gla', 'la'),
 (' lokalizacji', ' terenowych'),
 (' przejazdu', ' terroryzmu'),
 (' operacji', ' Schengen'),
 (' przejazdu', ' turystycznej'),
 ('ala', 'la'),
 (' lokalizacji', ' turystycznej'),
 (' oprogramowania', ' socjologii'),
 (' lokalizacji', ' zakwater'),
 (' przejazdu', ' policj'),
 (' nia', 'la'),
 (' inwestycji', ' turystycznej'),
 (' przejazdu', ' depor'),
 (' lokalizacji', ' spado'),
 ('scie', 'la'),
 (' inwestycji', ' dyplomatycznych'),
 (' aplikacji', ' s

Analiza Wvo i Wqk w przestrzeni osadzeń jak analiza ukrytych stanów lub wag funkcji Feed Forward, mogą dostarczyć cennych informacji na temat działania modelu.